In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd

<h2>Settings</h2>

In [2]:
result_dir = ""

compiler_type_vms = ["asmble", "wasmer", "wavm"]

interpreter_type_vms = ["life", "wasmi", "wagon"]

# the list of enabled test names
test_names = ["snappy_compression_5_1000000_1Kb", \
              "snappy_compression_5_10_100Mb", \
              "deflate_compression_5_100000_1Kb", \
              "deflate_compression_5_5_100Mb", \
              "fibonacci_38", \
              "factorization_2147483647", \
              "recursive_hash_10000000_0", \
              "matrix_product_1_10_1000000", \
              "matrix_product_1_500_100", \
              "svd_decomposition_1_10_1000000", \
              "svd_decomposition_1_300_100", \
              "qr_decomposition_1_10_1000000", \
              "qr_decomposition_1_500_100"]

<h2>Import results</h2>

In [3]:
from collections import defaultdict

def read_results(file_name):
    results = defaultdict(list)
    csv_results = pd.read_csv(file_name)
    for test_name, elapsed_time in zip(csv_results["test_name"], csv_results["elapsed_time"]):
        results[test_name].append(elapsed_time)
        
    return results

In [4]:
from os.path import join

vm_results = {}
for compiler_vm in compiler_type_vms:
    vm_results[compiler_vm] = read_results(join(result_dir, compiler_vm + ".csv"))

for interpreter_vm in interpreter_type_vms:
    vm_results[interpreter_vm] = read_results(join(result_dir, interpreter_vm + ".csv"))

KeyError: 'test_name'

<h2>Results preparation</h2>

In [ ]:
tests_result = defaultdict(list)
df_results = {}
interpreter_tests_result = {}

for test_name in test_names:
    for vm_name in vm_results:
        tests_result[test_name].append(vm_results[vm_name][test_name])
        
    df_results[test_name] = pd.DataFrame(pd.DataFrame(tests_result[test_name]).T.values, columns=vm_results.keys())

<h2>Result statistics</h2>

In [ ]:
for test_name in test_names:
    print("{}\n{}\n".format(test_name, df_results[test_name].describe()))

<h2>Box plots</h2>

In [ ]:
fontsize=15

def adjust_line2D_linewidth(line2d, box_line_width, whisker_line_width, cap_line_width, median_line_width):
    for box in line2d['boxes']:
        box.set(linewidth=box_line_width)
    
    for whisker in line2d['whiskers']:
        whisker.set(linewidth=whisker_line_width)

    for cap in line2d['caps']:
        cap.set(linewidth=cap_line_width)

    for median in line2d['medians']:
        median.set(linewidth=median_line_width)
        
def draw_plotbox(df, column, figsize, test_name, fontsize):
    lines = df.boxplot(figsize=figsize, column=column, return_type='dict', widths=0.5, fontsize=fontsize, showmeans=True)
    adjust_line2D_linewidth(lines, 2, 1, 1, 1)
    plt.title(test_name, fontsize=fontsize)
    plt.ylabel("sec", fontsize=fontsize)
    plt.show()
    
def draw_plot(df, columns, figsize, test_name, fontsize, isLogY=False):
    df[columns].plot(style="o-", figsize=figsize, fontsize=fontsize)
    plt.title(test_name, fontsize=fontsize)
    plt.ylabel("sec", fontsize=fontsize)
    if isLogY:
        plt.yscale("log")
    plt.grid()


<h3>snappy_compression_5_1000000_1Kb</h3>

In [ ]:
plot_test_name="snappy_compression_5_1000000_1Kb"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>snappy_compression_5_10_100Mb</h3>

In [ ]:
plot_test_name="snappy_compression_5_10_100Mb"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>deflate_compression_5_100000_1Kb</h3>

In [ ]:
plot_test_name="deflate_compression_5_100000_1Kb"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>deflate_compression_5_5_100Mb</h3>

In [ ]:
plot_test_name="deflate_compression_5_5_100Mb"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>fibonacci_38</h3>

In [ ]:
plot_test_name="fibonacci_38"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>factorization_2147483647</h3>

In [ ]:
plot_test_name="factorization_2147483647"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>recursive_hash_10000000_0</h3>

In [ ]:
plot_test_name="recursive_hash_10000000_0"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>matrix_product_1_10_1000000</h3>

In [ ]:
plot_test_name="matrix_product_1_10_1000000"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>matrix_product_1_500_100</h3>

In [ ]:
plot_test_name="matrix_product_1_500_100"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>svd_decomposition_1_10_1000000</h3>

In [ ]:
plot_test_name="svd_decomposition_1_10_1000000"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>svd_decomposition_1_300_100</h3>

In [ ]:
plot_test_name="svd_decomposition_1_300_100"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>qr_decomposition_1_10_1000000</h3>

In [ ]:
plot_test_name="qr_decomposition_1_10_1000000"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

<h3>qr_decomposition_1_500_100</h3>

In [ ]:
plot_test_name="qr_decomposition_1_500_100"
figsize=(12,10)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms + interpreter_type_vms, figsize, plot_test_name + " (all VMs)", fontsize, True)

In [ ]:
draw_plotbox(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name], compiler_type_vms, figsize, plot_test_name + " (compiler-type VMs)", fontsize)

In [ ]:
draw_plotbox(df_results[plot_test_name], interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)

In [ ]:
draw_plot(df_results[plot_test_name],  interpreter_type_vms, figsize, plot_test_name + " (interpreter-type VMs)", fontsize)